# Transform and Load
Brittany Lassiter

## Imports

In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

In [5]:
df = pd.read_csv('Data/final_tmdb_data_2010.csv.gz')
df.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0332280,0.0,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,NaN,29000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036.0,en,The Notebook,...,2004-06-25,115603229.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,0.0,7.88,10689.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2 non-null      object 
 1   adult                  1 non-null      float64
 2   backdrop_path          1 non-null      object 
 3   belongs_to_collection  0 non-null      float64
 4   budget                 1 non-null      float64
 5   genres                 1 non-null      object 
 6   homepage               1 non-null      object 
 7   id                     1 non-null      float64
 8   original_language      1 non-null      object 
 9   original_title         1 non-null      object 
 10  overview               1 non-null      object 
 11  popularity             1 non-null      float64
 12  poster_path            1 non-null      object 
 13  production_companies   1 non-null      object 
 14  production_countries   1 non-null      object 
 15  release_da